In [3]:
import pandas as pd 

In [5]:
# Import meta deta
meta_df = pd.read_csv("output/albania_metadata.csv")

In [6]:
meta_df.columns

Index(['variable_name', 'variable_description', 'module_name',
       'module_description', 'data_type', 'unique_id'],
      dtype='object')

In [7]:
meta_df['unique_id'].value_counts()

unique_id
id        472
psu-hh    270
hhid       76
hh_key      2
Name: count, dtype: int64

In [8]:
#Split metadata into groups to combine to create household and individual datasets
grouped = meta_df.groupby(["unique_id"])
ind_df = grouped.get_group(("id"))
h_df = grouped.get_group(("hhid"))
proxy_h_df = grouped.get_group(("psu-hh"))
map_df = grouped.get_group(("hh_key"))

C:\Users\PB\AppData\Local\Temp\ipykernel_8960\819969435.py:3: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  ind_df = grouped.get_group(("id"))
C:\Users\PB\AppData\Local\Temp\ipykernel_8960\819969435.py:4: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  h_df = grouped.get_group(("hhid"))
C:\Users\PB\AppData\Local\Temp\ipykernel_8960\819969435.py:5: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  proxy_h_df = grouped.get_group(("psu-hh"))
C:\Users\PB\AppData\Local\Temp\ipykernel_8960\819969435.py:6: FutureWarning: When grouping with a length

In [9]:
#Start with household data
h_df['module_name'].value_counts()

module_name
Modul_3_internet            24
Modul_1A_householdroster    22
poverty                     16
Modul_1B_householdroster    14
Name: count, dtype: int64